In [1]:
import pandas as pd
import os
import glob
import json
from tqdm.notebook import tqdm

In [2]:
FILE_PATH = os.getcwd() + r'\raw_json_data' #json file directory
files = glob.glob(os.path.join(FILE_PATH, '*.json'))
print(files)

['C:\\Users\\Kislee\\Workspace\\2023\\text_bias_analysis\\raw_json_data\\talksets-train-1.json', 'C:\\Users\\Kislee\\Workspace\\2023\\text_bias_analysis\\raw_json_data\\talksets-train-2.json', 'C:\\Users\\Kislee\\Workspace\\2023\\text_bias_analysis\\raw_json_data\\talksets-train-3.json', 'C:\\Users\\Kislee\\Workspace\\2023\\text_bias_analysis\\raw_json_data\\talksets-train-4.json', 'C:\\Users\\Kislee\\Workspace\\2023\\text_bias_analysis\\raw_json_data\\talksets-train-5.json', 'C:\\Users\\Kislee\\Workspace\\2023\\text_bias_analysis\\raw_json_data\\talksets-train-6.json']


In [3]:
dfs=[]
for filename in tqdm(files):
    with open(filename, 'r', encoding='utf-8') as f:
        js = json.loads(f.read(), strict=False)
        dfs.append(pd.json_normalize(data=js, record_path = 'sentences'))       
js_df = pd.concat(dfs)

  0%|          | 0/6 [00:00<?, ?it/s]

In [4]:
js_df.shape

(408369, 11)

In [5]:
js_df = js_df[['text', 'types', 'intensity']]

In [6]:
js_df.head()

,text,types,intensity
0,부랴부랴 왔는데 아무도 안왔네. 시간개념들이 없네,[CENSURE],2.0
1,맞아. 사람들이 진짜 개념이없다,[CENSURE],2.0
2,저렇게는 살지 말아야지,[CENSURE],2.0
3,그러게 게으르고 멍청한 사람들은 맞아야해,[CENSURE],2.0
4,인방 보는 남자는 거르는게 맞다,[CENSURE],1.0


In [7]:
# 데이터 타입 변경 
js_df = js_df.astype({'types':'string'})

In [8]:
# 비난, 혐오, 차별 데이터만 뽑아옴
df = js_df[(js_df['types'] == "['CENSURE']") | 
           (js_df['types'] == "['HATE']") | 
           (js_df['types'] == "['DISCRIMINATION']")]

In [9]:
df.shape

(114512, 3)

In [10]:
# 거슬리는 데이터 모양 바꾸기
df = df.replace({"['CENSURE']":'CENSURE',
                 "['HATE']":'HATE',
                 "['DISCRIMINATION']":'DISCRIMINATION'})

In [11]:
# 인덱스 초기화
df = df.reset_index(drop=True)

In [12]:
df.head()

,text,types,intensity
0,부랴부랴 왔는데 아무도 안왔네. 시간개념들이 없네,CENSURE,2.0
1,맞아. 사람들이 진짜 개념이없다,CENSURE,2.0
2,저렇게는 살지 말아야지,CENSURE,2.0
3,그러게 게으르고 멍청한 사람들은 맞아야해,CENSURE,2.0
4,인방 보는 남자는 거르는게 맞다,CENSURE,1.0


In [13]:
df.to_csv('data.csv')